In [46]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/Discrete_dynamical_system/Difference_Equation'
num_steps = 99
num_cases = 3
all_data = []
for case in range(num_cases):
    x = np.zeros(num_steps + 1)
    y = np.zeros(num_steps + 1)
    z = np.zeros(num_steps + 1)
    if case == 0:
        x[0] = 0.5
        y[0] = 1.5
        z[0] = 0
    elif case == 1:
        x[0] = 1.0
        y[0] = 0.5
        z[0] = 1
    elif case == 2:
        x[0] = 0.8
        y[0] = 1.2
        z[0] = 2
    for t in range(num_steps):
        x[t + 1] = y[t]
        y[t + 1] = (x[t] * z[t]) / (z[t] + 1) + 1 / y[t]
        z[t + 1] = z[t] + 1
    case_data = np.column_stack((x, y, z))
    all_data.append(case_data)
header = ['x1', 'x2', 'x3', 'trajectory']
with open('50.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for case in range(num_cases):
        trajectory = np.full((num_steps + 1, 1), case + 1)
        case_data_with_trajectory = np.column_stack((all_data[case], trajectory))
        writer.writerows(case_data_with_trajectory)
time_steps = np.arange(num_steps + 1)
def split_data():
    trajectories = {}
    column_names = None
    with open('../../../Data/Discrete_dynamical_system/Difference_Equation/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 4):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()